In [ ]:
#!/usr/bin/env python
# encoding: utf-8
import pyresample
import numpy
import numpy as np
import h5py
import re,os, glob, sys, getopt
from scipy.io import loadmat, savemat
from pyresample import image, geometry, kd_tree, utils
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import pyproj
import scipy
import netCDF4

def convert_rad_temp(rad,band=1):
    # convert rads counts to brightness temp in K
    goes13 = { # band 1 : [ Cente-Win, FK1, FK2, BC1, BC2
          1: [16285.9886,5.14481e+07,2.34319e+04, 0.00000,1.00000],
       2: [2563.9572, 2.00752e+05, 3.68896e+03, 1.47950, 0.99794],
       3: [1528.2016, 4.25079e+04, 2.19874e+03, 3.96963, 0.99112],
       4: [937.2177,  9.80503e+03, 1.34845e+03, 0.36348, 0.99876],
       6: [749.6309, 5.01730e+03, 1.07855e+03, 0.09497, 0.99960]}
    
    #temp  =  [ fk2 / (alog((fk1 / rad) + 1))  -  bc1 ] / bc2
    # rad = rad  =  fk1 / [ exp (fk2 / (bc1 + (bc2 * temp))) - 1 ]
    temp = ( goes13[band][2] / (numpy.log((goes13[band][1] / rad) + 1)) - goes13[band][3] ) / goes13[band][4]
    return temp 

def g_resample_data(lats, lons, data, proj_id='eqc', bounds=None):
    lats = numpy.array((lats))
    lons = numpy.array((lons))
    fillValue=999
    fillValue2=fillValue
    if  (lats.max() <= -89.5) or (lats.max() >= 89.5): 
        fillValue = lats.max()
    elif (lats.min() <= -89.5) or (lats.min() >= 89.5):
        fillValue = lats.min()
    
    mask = np.zeros(data.shape, dtype=bool)
    mask_range = np.where((lats >= fillValue ))
    mask[mask_range] = True
    lats = np.ma.masked_array(lats, mask)

    if (lons.max() <= -179.5) or (lons.max() >= 179.5):
        fillValue2 = lons.max()
    elif (lons.min() <= -179.5) or (lons.max() >= 179.5):
        fillValue2 = lons.min()
    
    mask = np.zeros(data.shape, dtype=bool)
    mask_range = np.where((lons >= fillValue2))
    mask[mask_range] = True
    lons = np.ma.masked_array(lons, mask)
    # print data.shape
    lats_median = np.median(lats)
    lons_median = np.median(lons)
    area_id = 'GOES Reprojection'
    area_name = area_id
    description = 'GOES Reprojection'
    # proj_id = 'eqc'
    # projection = '+proj=latlong +datum=WGS84 +ellps=WGS84 +lat_ts=%s +lat_0=%s +lon_0=%s +units=m +a=6371228.0' %(lats_median, lats_median, lons_median)
    x_size=lons.shape[1] # lon
    y_size=lons.shape[0]# lat
    area_dict = dict(datum='WGS84',lat_0=lats_median,lon_0=lons_median, proj=proj_id, units='m', a=6371228.0)
    if bounds == None:
        # x_size=lons.shape[1] # lon
        # y_size=lons.shape[0]# lat
        ur_lat=lats.max()
        ll_lat=lats.min()
        ur_lon=lons.max()
        ll_lon=lons.min()
    else:
       
        ur_lat=bounds['ur_lat']
        ll_lat=bounds['ll_lat']
        ur_lon=bounds['ur_lon']
        ll_lon=bounds['ll_lon']
        # compute grid dimension 
        x_size, y_size = compute_dims([ll_lon, ll_lat,ur_lon, ur_lat])
        
        
    lon_bbox=[ll_lon, ur_lon]
    lat_bbox=[ll_lat, ur_lat]
    print (lat_bbox, lon_bbox, lats.max(), lons.max(), lats.min(), lons.min())

    import pyproj
    prj=pyproj.Proj(area_dict)
    x, y = prj(lon_bbox, lat_bbox)
    area_extent = (x[0], y[0], x[1], y[1])
    # print area_extent
    # area_def = utils.get_area_def(area_id, description, proj_id, projection, x_size, y_size, area_extent)
    # use with area_dict
    area_def = geometry.AreaDefinition(area_id, area_name, proj_id, area_dict, x_size, y_size, area_extent)

    grid_def = geometry.GridDefinition(lons=lons, lats=lats)
    grid_con = image.ImageContainerNearest(data, grid_def, radius_of_influence=500000, epsilon=0.025)
    #grid_con = image.ImageContainerQuick(data, msg_area)
    area_con = grid_con.resample(area_def)
    result = area_con.image_data
    # result is resample to area aefinition, so lat lon is from area def
    result_lons, result_lats = area_def.get_lonlats()
    return area_def, result, result_lons, result_lats

def generate_quicklook_png(area_def, result, proj_id=None, outputname=None):

    if outputname == None:
        outputname = "some_random_XXX_"
    
    if proj_id == 'latlong':
        scipy.misc.imsave(outputname+'_full_res.png', result)
        n = scipy.misc.imresize(result, (126, 201),  'nearest', mode='F')
        print(n.max(), n.min())
        scipy.misc.imsave(outputname+'_full_res_201x126.png', n)
        numpy.array((result)).astype('f8').tofile(outputname+'.dat')
        return 
    # Quick lookup
    pyresample.plot.save_quicklook(outputname+'_quicklook.png', area_def, result, num_meridians=0, num_parallels=0)

    # on basemap blue marble
    bmap = pyresample.plot.area_def2basemap(area_def, resolution='h')
    bmng = bmap.bluemarble()
    bmap.drawmapboundary()
    bmap.drawcoastlines()
    bmap.drawstates()
    imb = bmap.imshow(result, origin='upper', cmap = plt.get_cmap('terrain'), alpha = 0.5)
    plt.savefig(outputname+'_terrain_rainbow.png', bbox_inches='tight')

    imb = bmap.imshow(result, origin='upper', cmap= plt.cm.gray)
    plt.savefig(outputname+'_gray.png', bbox_inches='tight')
    
    scipy.misc.imsave(outputname+'_full_res.png', result)
    print('saving out binary....')
    numpy.array((result)).astype('f8').tofile(outputname+'.dat')

def simple_map(data, outputname=None):
    if outputname==None:
        outputname='some_random_XXX_simple_'
    plt.imshow(data, origin='upper', cmap= plt.cm.gray)
    plt.savefig(outputname+'_gray.png')

def compute_dims(extent, resolution=[1, 1]):
    # Define KM_PER_DEGREE
    KM_PER_DEGREE = 111.32
    # Compute grid dimension
    sizex = int(((extent[2] - extent[0]) * KM_PER_DEGREE) / resolution[0])
    sizey = int(((extent[3] - extent[1]) * KM_PER_DEGREE) / resolution[1])
    return sizex, sizey

def process_granule(filename, units='radiance', outdir=None):
    band=int(filename.split('.')[4].split('_')[1])
    # print band, filename.strip('.nc').split('/')[-1]
    if outdir!=None and os.path.exists(outdir):
        output_filename = outdir +'/' + filename.strip('.nc').split('/')[-1]
    else:
        output_filename = filename.strip('.nc').split('/')[-1]
    print (output_filename)
    _data = netCDF4.Dataset(filename)
    _vars = _data.variables['lon']
    data = numpy.array(_data.variables['data'][:][0])
    # print data
    # goes data and band number
    if units == 'brightnesstemperature':
        data = convert_rad_temp(data,band=band)
    lon = numpy.array(_data.variables['lon'][:])
    lat = numpy.array(_data.variables['lat'][:])
    
    # bounds set by radar data (Brian V. Hall)
    bounds = { 'ur_lat' : 41,'ll_lat' : 31,'ur_lon' : -82, 'll_lon' : -102,}
    # bounds = { 'ur_lat' : 61,'ll_lat' : 11,'ur_lon' : -65, 'll_lon' : -112,}
    # bounds=None
    # print data
    area_def, result, result_lons, result_lats = g_resample_data(lat, lon, data, proj_id='laea', bounds=bounds)
    print (result.max(), result.min())
    generate_quicklook_png(area_def, result, proj_id='latlong', outputname=output_filename)
    # area_def, result = g_resample_data(lat, lon, data, proj_id='latlong', bounds=None)
    simple_map(data, outputname=output_filename+'unprocessed_')

if __name__=="__main__":
    #filename='goes13.2017.186.184519.BAND_01.nc'
    #filename='goes13.2017.230.171519.BAND_01.nc'
    filename='/Users/satyamsharma/Documents/CAPSTONE/Project/DATA/satellite/goes13.2017.182.001519.BAND_01.nc'
    #filename='goes13.2017.230.171519.BAND_01.nc'
    process_granule(filename, outdir=None, units='radiance')
    # process_granule(filename, outdir=None, units='brightnesstemperature')
